<a href="https://colab.research.google.com/github/jggomez/spark-demo/blob/main/Spark_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install the libraries**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# **Create a session**

In [14]:
from pyspark.sql import SparkSession, Row
from datetime import date

In [4]:
spark_session = SparkSession \
  .builder \
  .appName("dataframe_examples") \
  .config("spark.jars", "/content/neo4j-connector-apache-spark_2.12-4.1.5_for_spark_3.jar, /content/spark-3.3-bigquery-0.29.0-preview.jar") \
  .getOrCreate()


# **Spark Dataframe**

In [39]:
df_example = spark_session.createDataFrame([
    (11222, "Juan Guillermo", 38, date(1984, 8, 21), "CALI"),
    (33333, "Veronica", 45, date(1982, 8, 21), "BOGOTA"),
    (4444, "Miguel", 30, date(1975, 8, 21), "CALI")],
    schema="userid bigint, name string, age int, birthdate date, city string")

df_example.show()
df_example.printSchema()
df_example.show(1)
df_example.columns

+------+--------------+---+----------+------+
|userid|          name|age| birthdate|  city|
+------+--------------+---+----------+------+
| 11222|Juan Guillermo| 38|1984-08-21|  CALI|
| 33333|      Veronica| 45|1982-08-21|BOGOTA|
|  4444|        Miguel| 30|1975-08-21|  CALI|
+------+--------------+---+----------+------+

root
 |-- userid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- city: string (nullable = true)

+------+--------------+---+----------+----+
|userid|          name|age| birthdate|city|
+------+--------------+---+----------+----+
| 11222|Juan Guillermo| 38|1984-08-21|CALI|
+------+--------------+---+----------+----+
only showing top 1 row



['userid', 'name', 'age', 'birthdate', 'city']

In [40]:
df_example.select("age").describe().show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 3|
|   mean|37.666666666666664|
| stddev| 7.505553499465135|
|    min|                30|
|    max|                45|
+-------+------------------+



In [41]:
df_example.collect()

[Row(userid=11222, name='Juan Guillermo', age=38, birthdate=datetime.date(1984, 8, 21), city='CALI'),
 Row(userid=33333, name='Veronica', age=45, birthdate=datetime.date(1982, 8, 21), city='BOGOTA'),
 Row(userid=4444, name='Miguel', age=30, birthdate=datetime.date(1975, 8, 21), city='CALI')]

In [42]:
df_example.take(2)

[Row(userid=11222, name='Juan Guillermo', age=38, birthdate=datetime.date(1984, 8, 21), city='CALI'),
 Row(userid=33333, name='Veronica', age=45, birthdate=datetime.date(1982, 8, 21), city='BOGOTA')]

In [43]:
df_example.filter(df_example.userid == 11222).show()

+------+--------------+---+----------+----+
|userid|          name|age| birthdate|city|
+------+--------------+---+----------+----+
| 11222|Juan Guillermo| 38|1984-08-21|CALI|
+------+--------------+---+----------+----+



In [45]:
df_example.groupby("city").sum().show()

+------+-----------+--------+
|  city|sum(userid)|sum(age)|
+------+-----------+--------+
|  CALI|      15666|      68|
|BOGOTA|      33333|      45|
+------+-----------+--------+



# **Read a CSV**

In [61]:
df_user = spark_session.read.csv("users.csv", header=True, sep=",")
df_user.select("id").show()

+--------------------+
|                  id|
+--------------------+
|RAX2yfecEFOypF9M8...|
|YKloSHEXzpdVOMz7C...|
|5qj2wd3QEARg2eCoi...|
|LfWXCt6rV3d1n6OZy...|
|h78bKjUS8gcCo1Dd6...|
|XCv3X0JeVrTMwnfFq...|
|Z0s5GycXYEPqJjAro...|
|BSXKShCNWsaDeloMr...|
|pi47MxM1k1X08jVxu...|
|tGmgqvIP6JgDDbnx0...|
|VaET4sCISxe1ysbSj...|
|yOVLW562H0fk3jxcP...|
|bv26oR8yaZbrcxPxq...|
|IZBIy7qJtmhyz6o0v...|
|DV4H9VZG0TXnLxuvh...|
|ph1taUTwnNPAwkfQy...|
|MLt1fE1efIYZkUa9V...|
|oa4DtLbANEbo6LfKq...|
|oMUlAlMUOVPZPuM0i...|
|5K8V86p6ZkU7Ul4yL...|
+--------------------+
only showing top 20 rows



# **Filter**

In [63]:
df_user_mexico = df_user.filter((df_user.country == "MX") & (df_user.trialexpired == True))
df_user.select("id", "country", "trialexpired").show()

+--------------------+-------+------------+
|                  id|country|trialexpired|
+--------------------+-------+------------+
|RAX2yfecEFOypF9M8...|No data|     No data|
|YKloSHEXzpdVOMz7C...|No data|       false|
|5qj2wd3QEARg2eCoi...|No data|     No data|
|LfWXCt6rV3d1n6OZy...|No data|     No data|
|h78bKjUS8gcCo1Dd6...|No data|     No data|
|XCv3X0JeVrTMwnfFq...|No data|       false|
|Z0s5GycXYEPqJjAro...|     EC|     No data|
|BSXKShCNWsaDeloMr...|No data|     No data|
|pi47MxM1k1X08jVxu...|No data|     No data|
|tGmgqvIP6JgDDbnx0...|No data|     No data|
|VaET4sCISxe1ysbSj...|No data|     No data|
|yOVLW562H0fk3jxcP...|No data|     No data|
|bv26oR8yaZbrcxPxq...|No data|       false|
|IZBIy7qJtmhyz6o0v...|No data|     No data|
|DV4H9VZG0TXnLxuvh...|No data|     No data|
|ph1taUTwnNPAwkfQy...|No data|     No data|
|MLt1fE1efIYZkUa9V...|No data|     No data|
|oa4DtLbANEbo6LfKq...|No data|     No data|
|oMUlAlMUOVPZPuM0i...|No data|     No data|
|5K8V86p6ZkU7Ul4yL...|No data|  

# **Using SQL**

In [56]:
df_user_mexico.createOrReplaceTempView("users_mexico")
users_mexico_sql =spark_session.sql("SELECT count(*) FROM users_mexico")
users_mexico_sql.show()

+--------+
|count(1)|
+--------+
|    1104|
+--------+

